![alt text](Python10-OpenCV_extra/logos.jpeg)

Сегодня речь пойдёт про обработку изображений в python. Тема необъятная, но какие-то основы мы разберём. Помимо работы с картинками, мы посмотрим как на коленке обучить распознавание цифр и в несколько строк кода запустить детектор лиц с вебкамеры.

# Pillow

Pillow - это Python Imaging Library (PIL). Библиотека, добавляющая простую поддержку изображений: открыть, манипулировать, сохранить. 
Установка:
pip install pillow 

In [ ]:
from PIL import Image

In [ ]:
my_pil_image = Image.open("Python10-OpenCV_extra/what_is_pillow.jpeg")

my_pil_image.show() # чтобы посмотреть в отдельном окне
display(my_pil_image) # чтобы посмотреть прямо тут 

In [ ]:
# Тип объекта
type(my_pil_image)

In [ ]:
# Посмотреть информацию о изображении
print("Format: {0}\nSize: {1}\nMode: {2}".format(my_pil_image.format, my_pil_image.size, my_pil_image.mode))

In [ ]:
# Чтобы не путать очередность высоты и ширины, можно вызывать их напрямую
print('width: {}, height: {}'.format(my_pil_image.width, my_pil_image.height))

In [ ]:
# Сохранить изображение
my_pil_image.save('Python10-OpenCV_extra/what_is_pillow.png', 'png') 
# Так можно сохраниить в одном любом популярном формате, например jpg или png (более 30 форматов)

Большая часть лично моего общения с PIL сводится именно к этим базовым действиям, которые легко, просто и быстро сделать. Что-то более сложное обычно делается другими методами. 

Дело в том что PIL рассматривает изображение именно как изображение, а не как массив пикселей с которыми было бы легко работать. 

Однако есть ещё кое-какие полезные функции.

In [ ]:
# Кроп
cropped = my_pil_image.crop((200, 200, 500, 350))
display(cropped)

In [ ]:
# Различные фильтры, например блюр
from PIL import ImageFilter 

blurred = cropped.filter(ImageFilter.BLUR)
display(blurred)

In [ ]:
# Оттенки серого
grayscale = cropped.convert('L') 
display(grayscale)

In [ ]:
# Поворот
rotated = cropped.rotate(180)
display(rotated)

In [ ]:
# Изменение размера
resized = cropped.resize((128,128))
display(resized)

In [ ]:
# Чтение по ссылке из интернета.
import requests

url = 'https://i.pinimg.com/originals/db/d0/79/dbd0790244b52e00d03c5d8be3b8d4d1.jpg'

try:
    resp = requests.get(url, stream=True).raw    
except requests.exceptions.RequestException as e:  
    sys.exit(1)

try:
    url_img = Image.open(resp).resize((300,300))
except IOError:
    print("Unable to open image")
    sys.exit(1)
    
display(url_img)

In [ ]:
# Рисование
from PIL import ImageDraw

idraw = ImageDraw.Draw(url_img)

idraw.rectangle((10, 70, 200, 100), fill='red')
idraw.text((30, 80), "No way you are tired!", 40)
display(url_img)

Рисовать можно много чего, множество свойств, детально посмотреть можно тут 
https://pillow.readthedocs.io/en/3.1.x/reference/ImageDraw.html

In [ ]:
# Склейка картинок
def img_concat(im1, im2):    
    # Создадим новое изображение с удвоенной высотой
    dst = Image.new('RGB', (im1.width, im1.height + im2.height)) 
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst

In [ ]:
merged_img = img_concat(grayscale, blurred)
display(merged_img)

На этом с PIL завершим.
Запомним вот что: 

* Очередность каналов: RGB
* Очередность размеров: ширина, высота
* Очередность координат в прямоугольнике: X1, Y1, X2, Y2

# Scikit-image

Scikit-image (skimage) - библиотека алгоритмов для обработки изображений, использующая numpy массивы для представления изображений.

Установка: pip install scikit-image

В отличии от PIL теперь мы имеем в качестве объекта массив размера (height, width, channel), что дает нам простой доступ к каждому пикселю. В качестве примера удобства numpy нашёл вот такой код, рисующий шахматную доску в четыре строки

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

check = np.zeros((8, 8))
check[::2, 1::2] = 1 # чётные
check[1::2, ::2] = 1 # нечётные
plt.imshow(check, cmap='gray') 

C matplotlib мы разобрались ранее, поэтому постараемся пользоваться им минималистично чтобы не отвлекаться от сегодняшней темы

In [ ]:
from skimage import io # Модуль чтобы читать-писать в разных форматах

In [ ]:
# Открыть изображение
my_image = io.imread('Python10-OpenCV_extra/I_know_python.jpeg')
plt.imshow(my_image) 

In [ ]:
print(type(my_image))

In [ ]:
# Отлично, посмотрим свойства как у numpy
print('dtype: {}; shape: {}'.format(my_image.dtype, my_image.shape))

In [ ]:
# Заглянем внутрь
my_image

Обратите внимание на нормировку - значения от 0 до 255. Skimage работает и с float, поэтому надо держать это в голове.

In [ ]:
from skimage import img_as_float # Можно было просто разделить на 255, но так надёжнее

my_image_float = img_as_float(my_image) 
my_image_float

In [ ]:
plt.imshow(my_image_float) 
# Понимание изображения не утрачено

In [ ]:
# Сохранение изображения/
io.imsave('Python10-OpenCV_extra/skimage_png_emaple.png', my_image)
# Указывать выходной формат не обязательно, он сам поймёт

In [ ]:
# Переведём изображение из PIL
skimage_image = np.array(my_pil_image)
plt.imshow(skimage_image) 

In [ ]:
# Переведём в PIL
pil_image = Image.fromarray(my_image)
display(pil_image)

Методы "открыть", "посмотреть" и "сохранить" неизменно лидируют по популярности. Но это можно было и в PIL. Теперь давайте разбираться зачем нам skimage.

In [ ]:
from skimage import filters # Множество простых фильтров под любые потребности

# Заблюрим изображение при помощи гауссова фильтра
my_image_gaussian = filters.gaussian(my_image, sigma=3)
plt.imshow(my_image_gaussian)

In [ ]:
from skimage import color # Любые трансформации цветов 

img_gray = color.rgb2gray(my_image)
#plt.imshow(img_gray)
plt.imshow(img_gray, cmap='gray')

In [ ]:
# Найдём на изображении контуры несколькими способами.
fig=plt.figure(figsize=(10, 10))
fig.add_subplot(2, 2, 1)
plt.imshow(filters.roberts(img_gray), cmap='gray') # Фильтр Робертса
fig.add_subplot(2, 2, 2)
plt.imshow(filters.prewitt(img_gray), cmap='gray') # Фильтр Прюитта
fig.add_subplot(2, 2, 3)
plt.imshow(filters.scharr(img_gray), cmap='gray') # Фильтр Щара (?)
fig.add_subplot(2, 2, 4)
plt.imshow(filters.sobel(img_gray), cmap='gray') # Фильтр Собеля
plt.show()

In [ ]:
# Прогоним изображение через фильтры по порогу, рассчитанному разными методами, реализованными в skimage
fig = filters.try_all_threshold(img_gray, figsize=(10, 10), verbose=False)

In [ ]:
from skimage import exposure # нелокальные фильтры (используют большую часть картинки или всю её)

fig=plt.figure(figsize=(10, 10))
fig.add_subplot(2, 3, 1)
plt.imshow(my_image)

# Гамма коррекция
fig.add_subplot(2, 3, 2)
plt.imshow(exposure.adjust_gamma(my_image, 2))

# Логарифмическая коррекция
fig.add_subplot(2, 3, 3)
plt.imshow(exposure.adjust_log(my_image, 0.7))

# Выравнивание гистрограмм
fig.add_subplot(2, 3, 4)
plt.imshow(exposure.equalize_hist(my_image))

# Адаптивное выравнивание гистограмм
fig.add_subplot(2, 3, 5)
plt.imshow(exposure.equalize_adapthist(my_image))

# Сигмоидальная настройка
fig.add_subplot(2, 3, 6)
plt.imshow(exposure.adjust_sigmoid(my_image))
plt.show()

In [ ]:
# Более сложные алгоритмы, нацеленные на широкий спектр задач. 
# Зачастую на выходе имеем не изображение, а набор признаков
from skimage import feature

In [ ]:
fig=plt.figure(figsize=(20, 30))
fig.add_subplot(1, 3, 1)
plt.imshow(img_gray, cmap='gray') 
fig.add_subplot(1, 3, 2)

# Детектор Кенни
plt.imshow(feature.canny(img_gray), cmap='gray') 
fig.add_subplot(1, 3, 3)

# Гистограммы ориентированных градиентов (HOG)
fd, hog_image = feature.hog(my_image, orientations=16, pixels_per_cell=(4, 4),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)
plt.imshow(hog_image) 
plt.show()

Давайте возьмём из модуля features каскадный классификатор, найдём его предобученные веса для объекта "фронтальное лицо", поищем лицо на нашей фотографии и построим прямоугольник

In [ ]:
from matplotlib.patches import Rectangle
from skimage import data 

В модуле data находятся разные сохранённые данные: предобученные алгоритмы, изображения, базы данных и прочее. Не будем на этом фокусироваться, потому что само по себе оно никогда не нужно, а когда вдруг понадобится - всегда есть ссылка. Зачастую этим модулем удобно пользоваться для воспроизведения примеров.

In [ ]:
my_image = io.imread('Python10-OpenCV_extra/I_know_python.jpeg')

trained_file = data.lbp_frontal_face_cascade_filename()

# Initialize the detector cascade.
detector = feature.Cascade(trained_file)
detected = detector.detect_multi_scale(img=my_image,
                                       scale_factor=1.05,
                                       step_ratio=1,
                                       min_size=(10, 10),
                                       max_size=(300, 300))
print(detected)

plt.imshow(my_image)
img_desc = plt.gca() # get current axis

for patch in detected:
    img_desc.add_patch(
        Rectangle(
            (patch['c'], patch['r']),
            patch['width'],
            patch['height'],
            fill=False,
            color='r',
            linewidth=2
        )
    )

plt.show()

Обратили внимание что прямоугольник задаётся не как в PIL?

Давайте рассмотрим ещё одну нетривиальную задачу, сначала испортим изображение случайным шумом, а потом попробуем его восстановить. Ривз, устал, давайте сменим изображение на более простое текстуру.

In [ ]:
# Модуль восстановления/улучшения изображений. Возьмём самые популярные:
from skimage.restoration import denoise_tv_chambolle, denoise_bilateral, denoise_wavelet

# Случайный шум (соль и перец), с ним и будем бороться
from skimage.util import random_noise 

In [ ]:
original = np.array(cropped)

sigma = 0.155
noisy = random_noise(original, var=sigma**2)

fig, ax = plt.subplots(nrows=2, ncols=4, figsize=(8, 5),
                       sharex=True, sharey=True)

plt.gray()

ax[0, 0].imshow(noisy)
ax[0, 0].axis('off')
ax[0, 0].set_title('Noisy')
ax[0, 1].imshow(denoise_tv_chambolle(noisy, weight=0.1, multichannel=True))
ax[0, 1].axis('off')
ax[0, 1].set_title('TV')
ax[0, 2].imshow(denoise_bilateral(noisy, sigma_color=0.05, sigma_spatial=15,
                multichannel=True))
ax[0, 2].axis('off')
ax[0, 2].set_title('Bilateral')
ax[0, 3].imshow(denoise_wavelet(noisy, multichannel=True, rescale_sigma=True))
ax[0, 3].axis('off')
ax[0, 3].set_title('Wavelet denoising')

ax[1, 1].imshow(denoise_tv_chambolle(noisy, weight=0.2, multichannel=True))
ax[1, 1].axis('off')
ax[1, 1].set_title('(more) TV')
ax[1, 2].imshow(denoise_bilateral(noisy, sigma_color=0.1, sigma_spatial=15,
                multichannel=True))
ax[1, 2].axis('off')
ax[1, 2].set_title('(more) Bilateral')
ax[1, 3].imshow(denoise_wavelet(noisy, multichannel=True, convert2ycbcr=True,
                                rescale_sigma=True))
ax[1, 3].axis('off')
ax[1, 3].set_title('Wavelet denoising\nin YCbCr colorspace')
ax[1, 0].imshow(original)
ax[1, 0].axis('off')
ax[1, 0].set_title('Original')

fig.tight_layout()

plt.show()

Кто-то, конечно, может сказать что восстанавливая изображение мы испортили его, но тут уж на вкус и цвет все метрики разные

Наш лимит времени на skimage исчерпан, и мы много чего не посмотрели и никак не можем этого успеть в рамках половины лекции. 
![no time](Python10-OpenCV_extra/no_time.jpg)
Что нам надо запомнить о skimage? В нём есть много алгоритмов различной обработки изображений, начиная от примитивных, заканчивая достаточно интеллектуальными:     
* Geometric transformations, 
* Color space manipulation,
* Analysis,
* Filtering, 
* Segmentation,
* Morphology,
* Feature detection, 
* more 

Достаточно много примеров можно найти на официальном сайте https://scikit-image.org/docs/dev/auto_examples/


# OpenCV

OpenCV - Open Source Computer Vision Library. 
Библиотека алгоритмов компьютерного зрения, обработки изображений и численных алгоритмов общего назначения с открытым кодом. Реализована на C/C++, также разрабатывается для других языков, в том числе python. Может свободно использоваться в любых целях, за исключением нескольких платных пакетов. Активно использует параллелизм, как CPU так и на GPU, и вообще достаточно активно внедряет всё новое в свою библиотеку.

На официальном сайте написано что у них около 50 тысяч пользователей в сообществе, и более 18 млн скачек библиотеки.

Установка не такая простая как это было всегда до этого, и если мы хотим чтобы библиотека имела полный функционал (например, можно было работать с вебкамерой, и видео в целом), то придётся заморочиться. Но чтобы просто попробовать - достаточно будет **pip install opencv-python**. Более правильные варианты установки лучше гуглить, потому что библиотека постоянно обновляется, и варианты установки могут розниться в зависимости от того, что именно вы хотите с ней делать.

In [ ]:
# Импортируем так
import cv2 

In [ ]:
# Проверим любимые команды
test_image = cv2.imread("Python10-OpenCV_extra/opencv.jpg")
(h, w, d) = test_image.shape 
# Сначала высота, потом ширина, дальше каналы
print("height={}, width={}, depth={}".format(h, w, d))

In [ ]:
cv2.imshow("Image", test_image)
cv2.waitKey(0)                  # ждёт любую кнопку, с задержкой 0 милисекунд
cv2.destroyAllWindows()         # закрывает все открытые ранее окна

In [ ]:
# В самом Jupyter работать будет только через matplotlib.
from matplotlib import pyplot as plt

plt.imshow(test_image)

Самые наблюдательные из вас, могут заметить, что цвет лица на меме слегка отдаёт голубизной. Дело тут не в не в том что мем дефектный, а в том что OpenCV более 20 лет, он был создан раньше чем был принят формат RGB. Ранее был формат BGR, и он сохранён в библиотеке. И если в самом OpenCV это не вызывает внутренних противоречий, то при совместной работе с другими пакетами, например matplotlib надо об этом не забывать, чтобы не расплескать синеву по данным. 

In [ ]:
test_img_RGB = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
plt.imshow(test_img_RGB)

К слову, то же самое можно было сделать и руками, вспомнив первые лекции

In [ ]:
test_img_another_RGB = test_image[:,:,::-1]
plt.imshow(test_img_another_RGB)

In [ ]:
print(type(test_image)) # Тут ничего нового
print(test_image) # Тут тоже

In [ ]:
cv2.imwrite('Python10-OpenCV_extra/new_img_opencv.jpg', test_img_RGB)

Не забываем что чтобы сделать RGB мы поменяли каналы, но это не значит что opencv передумал, он по прежнему думает что это BGR, и при записи учтёт это.

В OpenCV есть множество алгоритмов, схожих с skimage, фильтры, обработка, восстановление и т.д. Но основная направленность всё же на компьютерное зрение. Его давайте и рассмотрим на примерах. Решим великую задачу машинного обучения - распознавание цифр, при помощи OpenCV, разумеется. 

Продвинутые алгоритмы для работы с большими сложными базами вам рассказывают на курсе машинного обучения, а здесь мы попробуем насладиться простотой и обучиться распознавать простые цифры по одной картинке.

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt


train_image = cv2.imread('./Python10-OpenCV_extra/data/train.png') 
# Проведём предобработку как ранее пробовали на skimage
gray = cv2.cvtColor(train_image, cv2.COLOR_BGR2GRAY)      # переводим в серый
blur = cv2.GaussianBlur(gray, (5, 5), 0)                  # блюрим (чтобы "сгладить" изображение)
thresh = cv2.adaptiveThreshold(blur, 255, 1, 1, 11, 2)    # фильтруем по порогу

fig=plt.figure(figsize=(20, 20))
fig.add_subplot(4, 1, 1)
plt.imshow(train_image)
fig.add_subplot(4, 1, 2)
plt.imshow(gray, cmap='gray')
fig.add_subplot(4, 1, 3)
plt.imshow(blur, cmap='gray')
fig.add_subplot(4, 1, 4)
plt.imshow(thresh, cmap='gray')

In [ ]:
# Найдём контуры
contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
#print(contours)
#print(len(contours))      # должно быть 125  

In [ ]:
# Ну, раз ничего непонятно, то давайте просто посмотрим
train_image_copy = train_image.copy()
for cnt in contours:        
    [x, y, w, h] = cv2.boundingRect(cnt) # извлекаем левый верхний угол и размеры контуров
    # А вот чтобы построить прямоугольник - надо задавать два набора координат!
    cv2.rectangle(train_image_copy, (x, y), (x + w, y + h), (0, 0, 255), 2)
plt.imshow(train_image_copy, cmap='gray')

Теперь понятно, некоторые цифры получили несколько "сработок". Не беда, возьмём только самые большие.

In [ ]:
# Заведём пустые места для записи лэйблов 
samples = np.empty((0, 100), np.float32)
responses = []
keys = [i for i in range(48, 58)] # соответствуюет клавишам цифр от 0 до 9 на стандартной раскладке

In [ ]:
# Напишем простенький алгоритм интерактивной разметки
for cnt in contours:    
    if cv2.contourArea(cnt) > 50: # отсеиваем все малые контуры
        [x, y, w, h] = cv2.boundingRect(cnt)

        if h > 28: # отсеиваем "половинки" цифр
            train_image_copy = train_image.copy()
            cv2.rectangle(train_image_copy, (x, y), (x + w, y + h), (0, 0, 255), 2)            
            
            # Выводим новый контур
            cv2.imshow('norm', train_image_copy)
            
            # И ждём разметку для него с клавиатуры
            key = cv2.waitKey(0)
            
            # Сделаем кнопку "q" стоп-словом, на случай если надоест размечать
            if key == ord('q'):
                cv2.destroyAllWindows()
                break
            
            elif key in keys:
                # Понимаем и запоминаем разметку с клавиатуры
                responses.append(int(chr(key))) 
                
                # Находим соответствующий ему фрагмент изображения
                roi = thresh[y:y + h, x:x + w]
                
                # Для удобства - отнормируем на 10 пикселей
                roismall = cv2.resize(roi, (10, 10)) 
                sample = roismall.reshape((1, 100)) 
                
                # И тоже запомним
                samples = np.append(samples, sample, 0)                
                

responses = np.array(responses, np.float32)
responses = responses.reshape((responses.size, 1))
print ("training complete")

samples = np.float32(samples)
responses = np.float32(responses)

cv2.imwrite("./Python10-OpenCV_extra/data/train_result.png", train_image)
np.savetxt('./Python10-OpenCV_extra/data/generalsamples.data', samples)
np.savetxt('./Python10-OpenCV_extra/data/generalresponses.data', responses)

In [ ]:
# Обучим KNN на наших данных
model = cv2.ml.KNearest_create()
model.train(samples, cv2.ml.ROW_SAMPLE, responses)

In [ ]:
test_image = cv2.imread('./Python10-OpenCV_extra/data/test.png') 
# Проведём предобработку как ранее пробовали на skimage
gray = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)      # переводим в серый
blur = cv2.GaussianBlur(gray, (5, 5), 0)                  # блюрим (чтобы "сгладить" изображение)
thresh = cv2.adaptiveThreshold(blur, 255, 1, 1, 11, 2)    # фильтруем по порогу

fig=plt.figure(figsize=(20, 20))
fig.add_subplot(4, 1, 1)
plt.imshow(test_image)
fig.add_subplot(4, 1, 2)
plt.imshow(gray, cmap='gray')
fig.add_subplot(4, 1, 3)
plt.imshow(blur, cmap='gray')
fig.add_subplot(4, 1, 4)
plt.imshow(thresh, cmap='gray')

In [ ]:
# Создадим чистую картинку такого же размера, чтобы написать на ней результаты распознавания
out = np.zeros(test_image.shape, np.uint8)

In [ ]:
# Повторим поиск и отсев контуров
contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

for cnt in contours:
    if cv2.contourArea(cnt) > 50:
        [x, y, w, h] = cv2.boundingRect(cnt)
        if h > 28:
            cv2.rectangle(test_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            roi = thresh[y:y + h, x:x + w]
            roismall = cv2.resize(roi, (10, 10))
            roismall = roismall.reshape((1, 100))
            roismall = np.float32(roismall)
            
            # Ищем самый похожий фрагмент по размеченному трейну
            retval, results, neigh_resp, dists = model.findNearest(roismall, k=1)
            
            # напишем его на нашей чистой картинке на соответсвующей позиции
            string = str(int((results[0][0])))
            cv2.putText(out, string, (x, y + h), 0, 1, (0, 255, 0))

In [ ]:
# Ну всё, давайте смотреть что вышло
cv2.imshow('out', out)

key = cv2.waitKey(0)
if key == ord('q'):
    cv2.destroyAllWindows()    

Как видим, всё отлично распозналось, если я не ошибся при разметке и не заленился сделать хотя бы пару строк. Это произошло в основном потому что тест простой и очень похож на трейн. Но что делать если тест чуть сложнее? Ну, например, появился небольшой шум, а наш простенький алгоритм не очень устойчив. Нам ведь не хочется чтобы из-за нескольких новых пикселей на изображении наша система порушилась и точность упала в ноль? Давайте рассмотрим пару методов морфологических преобразований, они не решат всех проблем, но укажут направление, в котором можно сходить за решением получше.

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('Python10-OpenCV_extra/j.png')

# Эрозия - "подтачиваем" границы
kernel = np.ones((5,5),np.uint8) 
# Ядро. Проходим ним картинку как скользящим окном. Если условие выполняется, то это 1, если нет - 0
erosion = cv2.erode(img,kernel,iterations = 1)

# Расширение. Проводим операцию с тем же ядром. Если условие условие не выполняется - то 1
dilation = cv2.dilate(img,kernel,iterations = 1)

fig=plt.figure(figsize=(10, 10))

fig.add_subplot(1, 3, 1)
plt.imshow(img, cmap='gray') 

fig.add_subplot(1, 3, 2)
plt.imshow(erosion, cmap='gray') 

fig.add_subplot(1, 3, 3)
plt.imshow(dilation, cmap='gray') 
plt.show()

In [ ]:
img_opening = cv2.imread('Python10-OpenCV_extra/opening.png')
img_closing = cv2.imread('Python10-OpenCV_extra/closing.png')

# Открытие: эрозия + расширение
opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)

# Закрытие: расширение + эрозия
closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

fig=plt.figure(figsize=(10, 10))

fig.add_subplot(2, 2, 1)
plt.imshow(img_opening, cmap='gray') 

fig.add_subplot(2, 2, 2)
plt.imshow(opening, cmap='gray') 

fig.add_subplot(2, 2, 3)
plt.imshow(img_closing, cmap='gray') 

fig.add_subplot(2, 2, 4)
plt.imshow(closing, cmap='gray') 

plt.show()

## Обработка видео
Следующий пример будет по работе с видеопотоком. 
Давайте поработаем с видео, например поищем там лица.

In [ ]:
import cv2

my_video = "./Python10-OpenCV_extra/my_video.mp4"
video = cv2.VideoCapture(my_video)

In [ ]:
print(video)
print(type(video))

# Проверим размеры и скорость
print('Frame width: {}'.format(video.get(cv2.CAP_PROP_FRAME_WIDTH)))
print('Frame height: {}'.format(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print('FPS: {}'.format(video.get(cv2.CAP_PROP_FPS)))

Вот так обращаемся к видеопотоку, но не считываем, а только открываем. Считывать будем покадрово.

In [ ]:
while True:            
    ret, frame = video.read()  
    # ret отвечает за то есть ли из видеопотока какой-то return. Если нет то надо выходить из цикла.
    # frame - кадр
    
    if not ret or cv2.waitKey(40) == ord('q'):
        break  
        
    # ну а с кадром можно делать всё что и раньше мы делали с изображениями
    cv2.imshow('Video', frame)
   
# обязательно завершить работу с видео
video.release() 
cv2.destroyAllWindows()

Для того чтобы цикл не ждал от нас сиюсекундного нажатия клавиши для сравнения с заданным ключом выхода, как это было нам удобно раньше, сделаем ожидание нажатие кнопки с задержкой: cv2.waitKey(1).
Заметим, что в отпущенное видео нельзя зайти повторно, если не инициализировать его снова.
Чтобы видео шло с реалистичной скоростью, надо отрегулирповать задержку между кадрами. Обычно это 25 кадров в секунду, что соответствует 40 милисекундам на кадр, но если вы имеете дело со скоростной камерой, или старым видео, то надо об этом не забыть. В нашем случае, видео хоть и старое, но скачано с ютьюба и там непонятный препроцесссинг, будем считать что это 25 кадров.

Не будем лезть в машинное обучение, и возьмём такой же каскад как смотрели в skimage. А предобученную модель качнём с официального гитхаба opencv https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml

In [ ]:
face_cascade = cv2.CascadeClassifier('./Python10-OpenCV_extra/haarcascade_frontalface_default.xml')

In [ ]:
print(face_cascade)

In [ ]:
while True:            
    ret, frame = video.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1)
    
    # Draw rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
    if not ret or cv2.waitKey(40) == ord('q'):
        break          
    
    cv2.imshow('Video', frame)   

video.release() 
cv2.destroyAllWindows()


Сразу видим ряд проблем. Во-первых лица не всегда находятся, во-вторых много ложных срабатываний, в-третьих видео стало тормозить. Мы конечно можем поиграться параметрами: ограничить минимальный размер лица, сделать шаги каскада поменьше, убрать задержку между кадрами... но действительно хорошего качества таким простым алгоритмом добиться не удастся. Давайте пока отложим борьбу за качество, а вместо этого попробуем вместо видеопотока из файла подсунуть видеопоток с вебкамеры.

In [ ]:
import cv2 
my_camera = cv2.VideoCapture(0) 

0 здесь это порядковый номер порта в который подключена камера. У ноутбука тоже 0, а вот если в ноутбук воткнуть вебкамеру, или просто к компьютеру подключить несколько, то они как-то будут пронумерованы, возможно в порядке подключения

In [ ]:
print(my_camera) 
print('Frame width: {}'.format(my_camera.get(cv2.CAP_PROP_FRAME_WIDTH)))
print('Frame height: {}'.format(my_camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print('FPS: {}'.format(my_camera.get(cv2.CAP_PROP_FPS)))

Не слишком-то отличается от видео.

In [ ]:
while True:            
    ret, frame = my_camera.read() 
    
    if not ret or cv2.waitKey(40) == ord('q'):
        break  
       
    cv2.imshow('Webcam', frame)   
    
my_camera.release() 
cv2.destroyAllWindows()

Ну и раз есть фреймы, то на них мы можем что-то попробовать запустить. Например, наш детектор лиц.

In [ ]:
face_cascade = cv2.CascadeClassifier('./Python10-OpenCV_extra/haarcascade_frontalface_default.xml')
while True:            
    ret, frame = my_camera.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.1)
    
    # Draw rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
    if not ret or cv2.waitKey(10) == ord('q'):
        break          
    
    cv2.imshow('Video', frame)   

my_camera.release() 
cv2.destroyAllWindows()

Теперь, когда базовые вещи мы посмотрели, можно посмотреть кое-что чуть сложнее. Чтобы у вас не осталось ощущения что мы сегодня изучали и занимались какими-то не очень важными вещами, которые и рамку-то вокруг лица нарисовать не могут, я все же подготовил детектор лиц получше на основе opencv. Есть модуль .dnn содержащий нейросетевые решения (много разных). Но мы не будем пользоваться каким-то совсем готовым и внесенным в пакет, потому что я что-то сходу не нашёл подходящего модуля под такую самую задачу, а вместо этого (погуглим, найдём и) зайдём на официальный гитхаб этого модуля https://github.com/opencv/opencv/tree/master/samples/dnn/face_detector и найдём там необученный модуль на caffe. Дальше (опять погуглив) легко найдём множество гитхабов, которые эту сеть обучили и выложили веса, например https://github.com/gopinath-balu/computer_vision/tree/master/CAFFE_DNN

In [ ]:
import cv2 
import numpy as np

# Архитектура с гита opencv
configFile = "Python10-OpenCV_extra/deploy.prototxt.txt"

# Обученные веса
modelFile = "Python10-OpenCV_extra/res10_300x300_ssd_iter_140000.caffemodel"

# Считываем модулем cv2.dnn
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

my_camera = cv2.VideoCapture(0) 
while True:            
    ret, frame = my_camera.read() 
    
    if not ret or cv2.waitKey(50) == ord('q'):
        break      
    
    h, w = frame.shape[:2]
    
    # Полезная функция-помощник. Она вычитает из изображения среднее, масштабирует, и еще может менять каналы
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, 
    (300, 300), (104.0, 117.0, 123.0))
    
    # Подаём картинку в сеть
    net.setInput(blob)
    
    # Получаем инференс
    faces = net.forward()
    
    # Рисуем
    for i in range(faces.shape[2]):
            confidence = faces[0, 0, i, 2]
            # Фильтруем по уверенности по порогу
            if confidence > 0.5:
                box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x, y, x1, y1) = box.astype("int")
                cv2.rectangle(frame, (x, y), (x1, y1), (0, 0, 255), 2)
    
    cv2.imshow('Webcam', frame)   
    
my_camera.release() 
cv2.destroyAllWindows()

Ура, наконец всё классно заработало!
На этом этой оптимистической ноте лекция окончена. 

Небольшое заключение: **Skimage vs OpenCV**

* **Skimage** хорош тем что изначально питонский, поэтому не возникнет никаких проблем совместимости. В нём больше алгоритмов обработки изображений и многие говорят что для обработки он *мощнее*.
* **OpenCV** хорош встроенными алгоритмами компьютерного зрения, всяческим мульти-процессингом и большим сообществом разработчиков-пользователей.

# На этом наш курс лекций окончен, ура!

По поводу оценок:

    для зачёта нужно 40%
    
    для экзамена: 
        3 - 40%
        4 - 60%
        5 - 80%
        
Через неделю (в крайнем слуаче - через 2) мы устроим разбор домашних заданий. Решений мы показывать не будем, но на вопросы постараемся ответить. Все претензии, аппеляции, все организационные вопросы и прочее нужно спросить до этого дня или прям там. 

Пожалуйста проверьте свой */status* и убедитесь что там всё в порядке.

Позднее "**я не знал / я лежал в больнице / я был в другой стране / меня отчисляют если не сдам / я всё сделал но не посылал потому что не думал что мне это нужно**" и прочие причины уже никак не могут ни на что повлиять.